In [ ]:
import rmgpy
from rmgpy.data.rmg import RMGDatabase
from rmgpy.rmg.react import *
from rmgpy.reaction import Reaction
from rmgpy.molecule.molecule import Molecule
from rmgpy.molecule.resonance import *
from rmgpy.species import Species
from rmgpy.thermo.thermoengine import submit

from IPython.display import display

In [ ]:
family = 'R_Addition_MultipleBond'

In [ ]:
databasePath = rmgpy.settings['database.directory']

database = RMGDatabase()
database.load(
    path = databasePath,
    thermoLibraries = ['primaryThermoLibrary'],
    reactionLibraries = [],
    seedMechanisms = [],
    kineticsFamilies = [family],
)

for family in database.kinetics.families.itervalues():
    family.addKineticsRulesFromTrainingSet(thermoDatabase=database.thermo)
    family.fillKineticsRulesByAveragingUp(verbose=True)

In [ ]:
reactants = (
    Species().fromSMILES("c1ccccc1"),
    Species().fromSMILES("c1[c]cccc1"),
)
for r in reactants:
    submit(r)

In [ ]:
for r in reactants:
    for m in r.molecule:
        display(m)

In [ ]:
result = reactSpecies(reactants)

In [ ]:
for rxn0 in result:
    kinetics, source, entry, isForward = database.kinetics.families[rxn0.family].getKinetics(rxn0, templateLabels=rxn0.template, degeneracy=rxn0.degeneracy, returnAllKinetics=False)
    rxn0.kinetics = kinetics

In [ ]:
for r in result:
    display(r)
    print r.template
    print r.kinetics